In [1]:
.I /root/install/do-conf-tp-serial/include/

In [2]:
.L libepetra

In [3]:
#include "Epetra_SerialComm.h"

In [4]:
Epetra_SerialComm Comm;

In [5]:
int NumMyElements = 20;

In [6]:
#include "Epetra_Map.h"
#include "Epetra_CrsMatrix.h"
#include "Epetra_Vector.h"

In [7]:
Epetra_Map Map(-1, NumMyElements, 0, Comm);
int NumGlobalElements = Map.NumGlobalElements();

Epetra_CrsMatrix A(Copy, Map, 3);

double negOne = -1.0;
double posTwo = 2.0;
for (int i=0; i<NumMyElements; i++) {
    int GlobalRow = A.GRID(i); int RowLess1 = GlobalRow - 1; int RowPlus1 = GlobalRow + 1;

    if (RowLess1!=-1) A.InsertGlobalValues(GlobalRow, 1, &negOne, &RowLess1);
    if (RowPlus1!=NumGlobalElements) A.InsertGlobalValues(GlobalRow, 1, &negOne, &RowPlus1);
    A.InsertGlobalValues(GlobalRow, 1, &posTwo, &GlobalRow);
};

A.FillComplete();

(int) 0


In [8]:
std::cout << A << std::endl;

Epetra::CrsMatrix
Number of Global Rows        = 20
Number of Global Cols        = 20
Number of Global Diagonals   = 20
Number of Global Nonzeros    = 58
Global Maximum Num Entries   = 3

Number of My Rows        = 20
Number of My Cols        = 20
Number of My Diagonals   = 20
Number of My Nonzeros    = 58
My Maximum Num Entries   = 3

   Processor    Row Index    Col Index           Value     
       0             0             0                       2    
       0             0             1                      -1    
       0             1             0                      -1    
       0             1             1                       2    
       0             1             2                      -1    
       0             2             1                      -1    
       0             2             2                       2    
       0             2             3                      -1    
       0             3             2                      -1    
       0         

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f0940c56640


In [9]:
double
powerMethod (const Epetra_Operator& A,
             const int niters,
             const double tolerance)
{
  using std::cout;
  using std::endl;

  // An Operator doesn't have a Comm, but its domain Map does.
  const Epetra_Comm& comm = A.OperatorDomainMap ().Comm ();
  const int myRank = comm.MyPID ();

  // Create three vectors for iterating the power method.  Since the
  // power method computes z = A*q, q should be in the domain of A and
  // z should be in the range.  (Obviously the power method requires
  // that the domain and the range are equal, but it's a good idea to
  // get into the habit of thinking whether a particular vector
  // "belongs" in the domain or range of the matrix.)  The residual
  // vector "resid" is of course in the range of A.
  Epetra_Vector q (A.OperatorDomainMap ());
  Epetra_Vector z (A.OperatorRangeMap ());
  Epetra_Vector resid (A.OperatorRangeMap ());

  // Local error code for use below.
  int lclerr = 0;

  // Fill the iteration vector z with random numbers to start.  Don't
  // have grand expectations about the quality of our pseudorandom
  // number generator; this is usually good enough for eigensolvers.
  //
  // If this call fails, just let the code below finish before trying
  // to catch the error globally.  Ditto for other calls below.
  lclerr = z.Random ();

  // lambda: the current approximation of the eigenvalue of maximum magnitude.
  // normz: the 2-norm of the current iteration vector z.
  // residual: the 2-norm of the current residual vector "resid"
  double lambda = 0.0;
  double normz = 0.0;
  double residual = 0.0;

  const double zero = 0.0;
  const double one = 1.0;

  // How often to report progress in the power method.  Reporting
  // progress requires computing a residual which can be expensive.
  // However, if you don't compute the residual often enough, you
  // might keep iterating even after you've converged.
  const int reportFrequency = 10;

  // Do the power method, until the method has converged or the
  // maximum iteration count has been reached.
  for (int iter = 0; iter < niters; ++iter) {
    // If you feel confident that your code is correct, you may omit
    // everything having to do with lclerr, and just write the following:
    //
    // z.Norm2 (&normz);         // Compute the 2-norm of z
    // q.Scale (one / normz, z); // q := z / normz
    // A.Apply (q, z);           // z := A * q
    // q.Dot (z, &lambda);       // Approx. max eigenvalue

    lclerr = (lclerr == 0) ? z.Norm2 (&normz) : lclerr;
    lclerr = (lclerr == 0) ? q.Scale (one / normz, z) : lclerr;
    lclerr = (lclerr == 0) ? A.Apply (q, z) : lclerr;
    lclerr = (lclerr == 0) ? q.Dot (z, &lambda) : lclerr;

    // Compute and report the residual norm every reportFrequency
    // iterations, or if we've reached the maximum iteration count.
    if (iter % reportFrequency == 0 || iter + 1 == niters) {
      // If you feel confident that your code is correct, you may omit
      // everything having to do with lclerr, and just write the
      // following:
      //
      // resid.Update (one, z, -lambda, q, zero); // z := A*q - lambda*q
      // resid.Norm2 (&residual); // 2-norm of the residual vector

      lclerr = (lclerr == 0) ? resid.Update (one, z, -lambda, q, zero) : lclerr;
      lclerr = (lclerr == 0) ? resid.Norm2 (&residual) : lclerr;

      if (myRank == 0) {
        cout << "Iteration " << iter << ":" << endl
             << "- lambda = " << lambda << endl
             << "- ||A*q - lambda*q||_2 = " << residual << endl;
      }
    }
    if (residual < tolerance) {
      if (myRank == 0) {
        cout << "Converged after " << iter << " iterations" << endl;
      }
      break;
    } else if (iter + 1 == niters) {
      if (myRank == 0) {
        cout << "Failed to converge after " << niters << " iterations" << endl;
      }
      break;
    }
  }

  // If any process failed to insert at least one entry, throw.
  int gblerr = 0;
  (void) comm.MaxAll (&lclerr, &gblerr, 1);
  if (gblerr != 0) {
    throw std::runtime_error ("The power method failed in some way.");
  }

  return lambda;
}

In [14]:
powerMethod(A, 1500, 1e-8);

Iteration 0:
- lambda = 1.15518
- ||A*q - lambda*q||_2 = 1.36821
Iteration 10:
- lambda = 3.88722
- ||A*q - lambda*q||_2 = 0.100102
Iteration 20:
- lambda = 3.91479
- ||A*q - lambda*q||_2 = 0.0504842
Iteration 30:
- lambda = 3.92385
- ||A*q - lambda*q||_2 = 0.035795
Iteration 40:
- lambda = 3.92971
- ||A*q - lambda*q||_2 = 0.0328482
Iteration 50:
- lambda = 3.9352
- ||A*q - lambda*q||_2 = 0.0330358
Iteration 60:
- lambda = 3.94082
- ||A*q - lambda*q||_2 = 0.0335013
Iteration 70:
- lambda = 3.94652
- ||A*q - lambda*q||_2 = 0.0333828
Iteration 80:
- lambda = 3.95204
- ||A*q - lambda*q||_2 = 0.0324585
Iteration 90:
- lambda = 3.95713
- ||A*q - lambda*q||_2 = 0.0307704
Iteration 100:
- lambda = 3.96159
- ||A*q - lambda*q||_2 = 0.0284921
Iteration 110:
- lambda = 3.96534
- ||A*q - lambda*q||_2 = 0.0258478
Iteration 120:
- lambda = 3.96838
- ||A*q - lambda*q||_2 = 0.0230537
Iteration 130:
- lambda = 3.97076
- ||A*q - lambda*q||_2 = 0.0202839
Iteration 140:
- lambda = 3.97259
- ||A*q - lambda

(double) 3.9776617
